# GSDMA project from dataframes

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_POL_LEXICON
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
df = pd.read_csv('real_data_subset_for_test.csv', sep=',')

In [3]:
dataframe_viewer(df, rows=5, un_val='ID')

Rows : 112, columns : 19, Unique values on cols: {'ID': 46}


interactive(children=(IntSlider(value=5, description='rows', max=112, min=5, readout=False), IntSlider(value=1…

## Data splitting 
(not necessary just to test)

In [4]:
df_litho = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'Litho_top', 'Litho_base', 'Epais_intv', 'Lithology']]
df_litho.rename(columns={'Lithology':'Description'}, inplace=True)

df_ech = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'ID_ech', 'Type_ech', 'Top_intv', 'Base_intv', 
             'Epais_intv', 'Bnz', 'Toln', 'Pb', 'As', 'Cu', 'Ni']]
df_ech.rename(columns={'Top_intv':'Ech_top', 'Base_intv':'Ech_base'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## Create pollutants list from the dataframe columns name

In [5]:
pollutants = []
for i, c in enumerate(df.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        #print(f"{i}: {c}")
        pollutants.append(c)
print(pollutants)

['Bnz', 'Toln', 'Pb', 'As', 'Cu', 'Ni']


In [6]:
# adds a description in sample (TEST)
bh_id = '201'
df_ech['Description'] = np.nan
idx = df_ech.query(f"ID=='{bh_id}'").index
df_ech.loc[idx, 'Description'] = ['Sable', 'Limon']

/tmp/ipykernel_17238/113860215.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ech['Description'] = np.nan
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Create Boreholes and components from dataframes

In [7]:
data_dict = {'lithologies_data': df_litho, 'pollutants_data': df_ech}

In [8]:
boreholes_orm, components, link_intv_comp = boreholes_from_dataframe(data_dict, verbose=False,
                                                sample_type_col='Type_ech', diameter_col='Diam_for', 
                                                default_z=102, attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}})


Data Processing...
To continue, default diameter column has been created with value: 0.1 [m]

 BH_ID: '201'
0- Interval top=0.6, base=1.2
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.6, base=1.2
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.7, base=1.2
 - Interval components: [Component({'lithology': 'sable'}), Component({'benzène': 'vi', 'concentration': 0.72, 'unit': 'mg/kg MS'}), Component({'toluène': 'vs', 'concentration': 0.59, 'unit': 'mg/kg MS'}), Component({'plomb': 'vs', 'concentration': 79.0, 'unit': 'mg/kg MS'}), Component({'arsenic': 'vr', 'concentration': 11.0, 'unit': 'mg/kg MS'}), Component({'cuivre': 'vs', 'concentration': 21.0, 'unit': 'mg/kg MS'}), Component({'nickel': 'vs', 'concentration': 90.0, 'unit': 'mg/kg MS'})]

3- Interval top=2.0, base=2.4
 - Interval components: [Component({'lithology': 'limon'}), Component({'benzène': 'vs', 'concentration': 0.12, 'unit': 'mg/kg MS'}), Component({'tol

/home/yanathan/Projects/GSDMA/utils/orm.py:203: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))
/home/yanathan/Projects/GSDMA/utils/orm.py:203: SAWarning: relationship 'ComponentOrm.intervals' will copy column Interv

 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.0, base=0.6
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.0, base=0.6
 - Interval components: [Component({'lithology': 'remblais'})]

3- Interval top=0.7, base=1.2
 - Interval components: [Component({'benzène': 'vs', 'concentration': 0.14, 'unit': 'mg/kg MS'}), Component({'toluène': 'vr', 'concentration': 0.13, 'unit': 'mg/kg MS'}), Component({'plomb': 'vs', 'concentration': 130.0, 'unit': 'mg/kg MS'}), Component({'arsenic': 'vs', 'concentration': 22.0, 'unit': 'mg/kg MS'}), Component({'cuivre': 'vi', 'concentration': 150.0, 'unit': 'mg/kg MS'}), Component({'nickel': 'vi_', 'concentration': 4100.0, 'unit': 'mg/kg MS'})]

4- Interval top=2.0, base=2.5
 - Interval components: [Component({'benzène': 'vs', 'concentration': 0.2, 'unit': 'mg/kg MS'}), Component({'toluène': 'vr', 'concentration': 0.15, 'unit': 'mg/kg MS'}), Component({'plomb': 'vs', 'concentration': 56.0, 'uni

2- Interval top=0.0, base=0.2
 - Interval components: [Component({'lithology': 'limons'})]

3- Interval top=3.1, base=3.6
 - Interval components: [Component({'benzène': 'vi_', 'concentration': 11.0, 'unit': 'mg/kg MS'}), Component({'toluène': 'vs', 'concentration': 0.93, 'unit': 'mg/kg MS'}), Component({'plomb': 'vs', 'concentration': 250.0, 'unit': 'mg/kg MS'}), Component({'arsenic': 'vs', 'concentration': 13.0, 'unit': 'mg/kg MS'}), Component({'cuivre': 'vs', 'concentration': 23.0, 'unit': 'mg/kg MS'}), Component({'nickel': 'vs', 'concentration': 43.0, 'unit': 'mg/kg MS'})]

4- Interval top=5.5, base=6.0
 - Interval components: [Component({'benzène': 'vi_', 'concentration': 25.0, 'unit': 'mg/kg MS'}), Component({'toluène': 'vi', 'concentration': 41.0, 'unit': 'mg/kg MS'}), Component({'plomb': 'vr', 'concentration': 15.0, 'unit': 'mg/kg MS'}), Component({'arsenic': 'vr', 'concentration': 4.0, 'unit': 'mg/kg MS'}), Component({'cuivre': 'vr', 'concentration': 10.0, 'unit': 'mg/kg MS'}),


 BH_ID: 'F13M'
0- Interval top=0.0, base=0.3
 - Interval components: [Component({'lithology': 'alluvions'})]

1- Interval top=0.0, base=0.4
 - Interval components: [Component({'lithology': 'alluvions'})]

2- Interval top=0.0, base=2.4
 - Interval components: [Component({'lithology': 'alluvions'})]

3- Interval top=4.2, base=4.7
 - Interval components: [Component({'benzène': 'vr', 'concentration': 0.05, 'unit': 'mg/kg MS'}), Component({'toluène': 'vr', 'concentration': 0.05, 'unit': 'mg/kg MS'}), Component({'plomb': 'vr', 'concentration': 17.0, 'unit': 'mg/kg MS'}), Component({'arsenic': 'vr', 'concentration': 6.2, 'unit': 'mg/kg MS'}), Component({'cuivre': 'vr', 'concentration': 11.0, 'unit': 'mg/kg MS'}), Component({'nickel': 'vr', 'concentration': 21.0, 'unit': 'mg/kg MS'})]

4- Interval top=0.0, base=0.5
 - Interval components: [Component({'benzène': 'vr', 'concentration': 0.05, 'unit': 'mg/kg MS'}), Component({'toluène': 'vr', 'concentration': 0.05, 'unit': 'mg/kg MS'}), Component

In [9]:
len(boreholes_orm), len(components)

(46, 15)

In [10]:
boreholes_orm # list of core.BoreholeOrm objects

[<core.orm.BoreholeOrm>(id=201, length=1.7999999999999998, diameter=0.1, intervals=5),
 <core.orm.BoreholeOrm>(id=205, length=3.0, diameter=0.1, intervals=9),
 <core.orm.BoreholeOrm>(id=208, length=2.999999999999999, diameter=0.1, intervals=9),
 <core.orm.BoreholeOrm>(id=207, length=3.600000000000001, diameter=0.1, intervals=11),
 <core.orm.BoreholeOrm>(id=212, length=2.9999999999999982, diameter=0.1, intervals=9),
 <core.orm.BoreholeOrm>(id=214, length=3.399999999999999, diameter=0.1, intervals=9),
 <core.orm.BoreholeOrm>(id=217, length=3.000000000000001, diameter=0.1, intervals=9),
 <core.orm.BoreholeOrm>(id=221, length=0.8, diameter=0.1, intervals=3),
 <core.orm.BoreholeOrm>(id=223, length=1.6, diameter=0.1, intervals=5),
 <core.orm.BoreholeOrm>(id=?2, length=0.7999999999999998, diameter=0.1, intervals=3),
 <core.orm.BoreholeOrm>(id=?1, length=0.7999999999999998, diameter=0.1, intervals=3),
 <core.orm.BoreholeOrm>(id=224, length=1.0, diameter=0.1, intervals=3),
 <core.orm.BoreholeOr

In [11]:
components # dict of striplog.Component objects

{0: Component({'type': 'lithology', 'value': 'Forage'}),
 1: Component({'type': 'lithology', 'value': 'remblai'}),
 2: Component({'type': 'lithology', 'value': 'sable'}),
 3: Component({'type': 'pollutant', 'value': 'benzène'}),
 4: Component({'type': 'pollutant', 'value': 'toluène'}),
 5: Component({'type': 'pollutant', 'value': 'plomb'}),
 6: Component({'type': 'pollutant', 'value': 'arsenic'}),
 7: Component({'type': 'pollutant', 'value': 'cuivre'}),
 8: Component({'type': 'pollutant', 'value': 'nickel'}),
 9: Component({'type': 'lithology', 'value': 'limon'}),
 10: Component({'type': 'lithology', 'value': 'l'}),
 11: Component({'type': 'lithology', 'value': 'schiste'}),
 12: Component({'type': 'lithology', 'value': 'béton'}),
 13: Component({'type': 'lithology', 'value': 'alluvion'}),
 14: Component({'type': 'lithology', 'value': 'a'})}

In [12]:
boreholes_orm[0].intervals#_values

{0: <core.orm.IntervalOrm>(id=0, top=<core.orm.PositionOrm object at 0x7f00c015ba00>, base=<core.orm.PositionOrm object at 0x7f00c015b970>, description={'borehole_type':'Forage'}, components=[]),
 1: <core.orm.IntervalOrm>(id=1, top=<core.orm.PositionOrm object at 0x7f00c015b070>, base=<core.orm.PositionOrm object at 0x7f00c0128580>, description=Remblais avec briques de construction couleur orange, gris, verdâtre  , components=[]),
 2: <core.orm.IntervalOrm>(id=2, top=<core.orm.PositionOrm object at 0x7f00c01289d0>, base=<core.orm.PositionOrm object at 0x7f00c0128a60>, description=Remblais avec briques de construction couleur orange, gris, verdâtre  , components=[]),
 3: <core.orm.IntervalOrm>(id=3, top=<core.orm.PositionOrm object at 0x7f00c0128a90>, base=<core.orm.PositionOrm object at 0x7f00c0128b20>, description=Sable, components=[]),
 4: <core.orm.IntervalOrm>(id=4, top=<core.orm.PositionOrm object at 0x7f00c0128b50>, base=<core.orm.PositionOrm object at 0x7f00c0128bb0>, descripti

In [13]:
i = 0
components[i][list(components[i].keys())[0]]#.json()

'lithology'

## Create a dictionary containing legends for each representation attribute

In [14]:
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]+pollutants)

## Create a database for the project

In [15]:
DB_name = 'tmp_files/test_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=False)

In [16]:
Base.metadata.create_all(engine)

In [17]:
Session = sessionmaker(bind=engine)
session = Session()

## create a project

In [18]:
p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

### Insert boreholes and components into the project

In [19]:
p.add_components(components)

In [20]:
for bhorm in boreholes_orm:
    p.insert_borehole(bhorm)

In [21]:
p.add_link_components_intervals(link_intv_comp)

### Validate modifications 

In [22]:
p.commit()

In [23]:
boreholes_orm[0].intervals[0].components

[<core.orm.ComponentOrm>(id=0, description={'type': 'lithology', 'value': 'Forage'})]

In [24]:
p.refresh()

In [25]:
session.close()

## update project legend dict

In [26]:
p.update_legend_cmap(compute_all_attrib=True, legend_dict=legend_dict, verbose=False)

In [27]:
p.legend_dict#['lithology']['legend']

{'lithology': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00cea90>,
  'values': []},
 'benzène': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00d4fa0>,
  'values': []},
 'toluène': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00ceee0>,
  'values': []},
 'plomb': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00d4ac0>,
  'values': []},
 'arsenic': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00cefa0>,
  'values': []},
 'cuivre': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00d4c70>,
  'values': []},
 'nickel': {'legend': Legend(),
  'cmap': <matplotlib.colors.ListedColormap at 0x7f00c00d4cd0>,
  'values': []}}

## Visualization

In [28]:
attributes = list(legend_dict.keys())
attributes

['lithology', 'benzène', 'toluène', 'plomb', 'arsenic', 'cuivre', 'nickel']

In [29]:
n = 0
bh = 1
repr_att = attributes[n]

### Log plot for one borehole

In [30]:
p.boreholes_3d[0].intervals

IndexError: list index out of range

In [ ]:
p.boreholes_3d[n].attrib_components(attribute=repr_att)

In [ ]:
p.boreholes_3d[n].components 
# lacking properties in components, why ???

In [ ]:
p.boreholes_3d[n].plot_log(repr_attribute=repr_att)

### 3D display of one borehole

In [ ]:
at = 1
n = 5
repr_att = attributes[at]
list(p.boreholes_3d[n].legend_dict.keys())[at]

In [ ]:
p.legend_dict[repr_att]['cmap']

In [ ]:
p.boreholes_3d[n].geometry.data[at].array.array

In [ ]:
p.boreholes_3d[n].plot_3d(repr_attribute=repr_att, str_annotations=False)

In [ ]:
p.boreholes_3d[n]._vtk.CELL_DATA_FIELD #.get_array #geometry.data[2].array.array

In [ ]:
from core.visual import Borehole3D
from striplog import Interval, Component

In [ ]:
intv = [Interval(0,1, components=[Component({'lithology':'sable'})]), 
        Interval(1,2, components=[Component({'lithology':'limon'})]),
        Interval(2,3, components=[Component({'lithology':'argile'})]),
        Interval(3,4, components=[Component({'lithology':'inconnu'})])]

In [ ]:
bh_test = Borehole3D({'lithology':intv, 'sample':None}, repr_attribute='lithology', length=4, legend_dict=legend_dict)

In [ ]:
bh_test.plot_log()#3d()

In [ ]:
p.boreholes_3d[n]._vtk

### Localization of project boreholes

### 3D display of project boreholes

In [ ]:
pause

In [ ]:
p.plot_3d(repr_attribute=repr_att, labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)

## Testing zone